In [1]:
# import numpy
import numpy as np

# import auxiliary function to construct a QG model instance and an integrator instance 
from pyshqg.core_numpy.constructors import construct_model, construct_integrator

# import auxiliary function to load the test data
from pyshqg.preprocessing.reference_data import load_test_data

In [2]:
# define a function to compute the relative RMSE between two vectors
def relative_rmse(x, y):
    rms_x = np.sqrt(np.mean(np.square(x)))
    rms_y = np.sqrt(np.mean(np.square(y)))
    rms_xy = np.sqrt(np.mean(np.square(x-y)))
    return rms_xy, rms_x, rms_y, 2*rms_xy/(rms_x+rms_y)

In [3]:
# load the test data at T21 resolution, interpolated in the T31 grid
ds_test = load_test_data(
    internal_truncature=21,
    grid_truncature=31,
)

In [4]:
ds_test

<xarray.Dataset>
Dimensions:             (ensemble: 16, level: 3, lat: 32, lon: 64, c: 2, l: 22,
                         m: 22)
Coordinates:
  * lat                 (lat) float64 85.76 80.27 74.74 ... -74.74 -80.27 -85.76
  * lon                 (lon) float64 0.0 5.625 11.25 ... 343.1 348.8 354.4
Dimensions without coordinates: ensemble, level, c, l, m
Data variables: (12/21)
    dissip_ekman        (ensemble, level, lat, lon) float64 0.0 ... -1.265e-11
    dpsi_dphi           (ensemble, level, lat, lon) float64 -0.1937 ... -0.2856
    dpsi_dtheta         (ensemble, level, lat, lon) float64 -0.02461 ... 0.3131
    dq_dphi             (ensemble, level, lat, lon) float64 1.216e-12 ... -2....
    dq_dtheta           (ensemble, level, lat, lon) float64 -1.245e-13 ... 4....
    forcing             (level, lat, lon) float64 -7e-11 ... -1.593e-10
    ...                  ...
    spec_q              (ensemble, level, c, l, m) float64 -5.365e-07 ... -7....
    spec_rk2            (ensemble, level, c, l, m) float64 -5.364e-07 ... -7....
    spec_rk4            (ensemble, level, c, l, m) float64 -5.364e-07 ... -7....
    spec_tendencies     (ensemble, level, c, l, m) float64 2.901e-16 ... -2.3...
    spec_total_q        (ensemble, level, c, l, m) float64 -5.365e-07 ... -3....
    zeta                (ensemble, lat, lon) float64 3.467e-06 ... -2.833e-06
Attributes:
    config:   {'abm_integration': {'dt': 1800, 'method': 'abm'}, 'data_interp...

In [5]:
# create a model using the configuration in the test data
model = construct_model(ds_test.config)

In [6]:
# create the 4th order Runge--Kutta integrator using the configuration in the test data
rk4 = construct_integrator(ds_test.config['rk4_integration'], model)

In [7]:
# create a model state using the test data
state = model.model_state(ds_test.spec_q.to_numpy())

In [8]:
# `state` is now a dictionnary
print(type(state))
print(state.keys())

<class 'dict'>
dict_keys(['spec_q'])


In [9]:
# the spectral vorticity is stored at key "spec_q" as a numpy array
print(type(state['spec_q']))

<class 'numpy.ndarray'>


In [10]:
# the shape of "spec_q" is (ensemble_size, num_levels, 2, T+1, T+1)
print(state['spec_q'].shape)

(16, 3, 2, 22, 22)


In [11]:
# run one integration step
fwd_state = rk4.forward(state)

In [12]:
# compare the obtained state to the test data
# here the relative RMSE is lower than 2e-10
# so our integration should be correct
relative_rmse(
    fwd_state['spec_q'],
    ds_test.spec_rk4.to_numpy(),
)

(7.463234256549668e-16,
 4.2354746605770695e-06,
 4.235474660384873e-06,
 1.7620774186622474e-10)

In [13]:
# re-initialise the model state
state = model.model_state(ds_test.spec_q.to_numpy())

# compute a model trajectory using the integrator
# to do so, we need to specify:
# - the starting time;
# - the number of snapshots to compute;
# - the number of inegration steps between two snapshots
# - the variables that we want to record
traj = rk4.run(
    state,
    t_start=0,
    num_snapshots=8,
    num_steps_per_snapshot=2,
    variables=('spec_q', 'spec_psi', 'zeta', 'q', 'psi'),
)

# the result is return as a QGTrajectory object
print(type(traj))

model integration:   0%|          | 0/8 [00:00<?, ?it/s]

<class 'pyshqg.core_numpy.model.QGModelTrajectory'>


In [14]:
# the trajectory object can be converted into a dictionnary of numpy arrays
# or into an xarray.Dataset:
traj.to_xarray()

<xarray.Dataset>
Dimensions:   (time: 9, batch_0: 16, level: 3, c: 2, l: 22, m: 22, lat: 32,
               lon: 64)
Coordinates:
  * time      (time) int64 0 3600 7200 10800 14400 18000 21600 25200 28800
  * lat       (lat) float64 -85.76 -80.27 -74.74 -69.21 ... 74.74 80.27 85.76
  * lon       (lon) float64 180.0 185.6 191.2 196.9 ... 157.5 163.1 168.8 174.4
Dimensions without coordinates: batch_0, level, c, l, m
Data variables:
    spec_q    (time, batch_0, level, c, l, m) float64 -5.365e-07 ... -8.539e-09
    spec_psi  (time, batch_0, level, c, l, m) float64 0.0 0.0 ... 380.3 153.0
    zeta      (time, batch_0, lat, lon) float64 3.467e-06 ... -5.777e-06
    q         (time, batch_0, level, lat, lon) float64 -0.000298 ... -0.0001231
    psi       (time, batch_0, level, lat, lon) float64 1.445e+08 ... -1.807e+07